## Свёрточные нейронные сети и аугментация данных
В этом задании необходимо обучить конволюционную нейросеть типа сети LeNet5, предложенную Яном Лекуном в 1998 году, для распознавания рукописных цифр из базы данных MNIST.  
Код ниже предлагает базовый вариант обучения, который даёт около 98-99% точности на тестовой выборке.  
Необходимо чуть-чуть улучшить данный код, введя **аугментацию** данных, т.е. расширить обучающий датасет за счёт дополнительных искажений исходных картинок:  
 - небольших случайных поворотов (rotate), 
 - дрожания (jitter), 
 - кроппинга (crop) 
 - и др.   
Для трансформации картинок используйте методы библиотеки `torchvision.transforms`, про которые можно прочитать [здесь](https://pytorch.org/vision/stable/transforms.html)  
Целью задания является - получить точность на тестовой выборке 99.5%.


In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
from matplotlib import pyplot  as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

cpu


# 1 Этап - подготовка данных!

In [8]:
train_transform = transforms.Compose(
                    [
#                     transforms.ToPILImage(),
#                     transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
#                     transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    transforms.ToTensor(),
                    transforms.RandomRotation(20),
                    v2.ColorJitter(brightness=.2),
#                     v2.CenterCrop(50),
                    v2.RandomPerspective(distortion_scale=0.2, p=0.3),
#                     v2.ElasticTransform(alpha=70.0),
                    ])

In [9]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=train_transform,  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# 2 Этап - Нейронная сеть!

<img src='Lenet5.png'>

In [10]:
# Fully connected neural network with one hidden layer
class Lenet5(nn.Module):
    def __init__(self, hidden_size1=500, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=0)
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1, padding=0)
        self.batchnorm2 = nn.BatchNorm2d(50)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(50 * 4 * 4, hidden_size1) # fully connected
        self.fc2 = nn.Linear(hidden_size1, num_classes)  
    
    def forward(self, x): # вход размера 1 x 28 x 28
        out = self.conv1(x) # на выходе размер 20 x 24 x 24
        out = self.batchnorm1(out)
        out = self.maxpool(out) # на выходе размер 20 x 12 x 12
        out = self.relu(out)
        out = self.conv2(out)  # на выходе размер 50 x 8 x 8
        out = self.batchnorm2(out)
        out = self.maxpool(out) # на выходе размер 50 x 4 x 4
        out = self.relu(out)
        out = out.reshape(-1, 50 * 4 * 4) # на выходе размер 800
        out = self.fc1(out) # на выходе размер 500
        out = self.relu(out)
        out = self.fc2(out) # на выходе размер 10
        
        return out

model = Lenet5(hidden_size1=500, num_classes=10).to(device)


# 3 этап - Loss-функция и оптимизатор

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 4 этап - обучение нейросети

Без трансформеров:

In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1745
Epoch [1/5], Step [200/600], Loss: 0.0559
Epoch [1/5], Step [300/600], Loss: 0.0214
Epoch [1/5], Step [400/600], Loss: 0.0899
Epoch [1/5], Step [500/600], Loss: 0.0714
Epoch [1/5], Step [600/600], Loss: 0.0412
Accuracy of the network on the 10000 test images: 98.49 %
Epoch [2/5], Step [100/600], Loss: 0.0645
Epoch [2/5], Step [200/600], Loss: 0.0974
Epoch [2/5], Step [300/600], Loss: 0.0042
Epoch [2/5], Step [400/600], Loss: 0.0151
Epoch [2/5], Step [500/600], Loss: 0.0118
Epoch [2/5], Step [600/600], Loss: 0.0051
Accuracy of the network on the 10000 test images: 98.97 %
Epoch [3/5], Step [100/600], Loss: 0.0581
Epoch [3/5], Step [200/600], Loss: 0.0270
Epoch [3/5], Step [300/600], Loss: 0.0576
Epoch [3/5], Step [400/600], Loss: 0.0005
Epoch [3/5], Step [500/600], Loss: 0.0841
Epoch [3/5], Step [600/600], Loss: 0.0127
Accuracy of the network on the 10000 test images: 99.01 %
Epoch [4/5], Step [100/600], Loss: 0.0069
Epoch [4/5], Step [200/600],

С трансформерами:

In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.2197
Epoch [1/5], Step [200/600], Loss: 0.0434
Epoch [1/5], Step [300/600], Loss: 0.0787
Epoch [1/5], Step [400/600], Loss: 0.0386
Epoch [1/5], Step [500/600], Loss: 0.0654
Epoch [1/5], Step [600/600], Loss: 0.0877
Accuracy of the network on the 10000 test images: 98.74 %
Epoch [2/5], Step [100/600], Loss: 0.0487
Epoch [2/5], Step [200/600], Loss: 0.0680
Epoch [2/5], Step [300/600], Loss: 0.0117
Epoch [2/5], Step [400/600], Loss: 0.0593
Epoch [2/5], Step [500/600], Loss: 0.0526
Epoch [2/5], Step [600/600], Loss: 0.0125
Accuracy of the network on the 10000 test images: 99.13 %
Epoch [3/5], Step [100/600], Loss: 0.0424
Epoch [3/5], Step [200/600], Loss: 0.0446
Epoch [3/5], Step [300/600], Loss: 0.0475
Epoch [3/5], Step [400/600], Loss: 0.0078
Epoch [3/5], Step [500/600], Loss: 0.0763
Epoch [3/5], Step [600/600], Loss: 0.0609
Accuracy of the network on the 10000 test images: 99.06 %
Epoch [4/5], Step [100/600], Loss: 0.0621
Epoch [4/5], Step [200/600],